# Amadeus Data Science Hiring Test Report

This notebook will be my report for the hiring test of Amadeus. For further infrmation, do not hesitate to contact me at [louis.auneau@amadeus.com](mailto:louis.auneau@amadeus.com).

## Abstract

Todo

## Content

[1. Introduction](#1.-Introduction)  
[2. Methods & results](#2.-Methods-&-results)  
...... [2.1 Data collection](#2.1-Data-collection)  
...... [2.2 Data cleaning and pre-processing](#2.2-Data-cleaning-and-pre-processing)  
...... [2.3 Descriptive Analysis](#2.3-Descriptive-Analysis)  
...... [2.4 Predictive Models](#2.4-Predictive-Models)  
............ [2.4.1 ARIMA](#2.4.1-ARIMA)  
............ [2.4.2 Random Forests](#2.4.2-Random-Forests)  
[3. Discussion and feedback](#3.-Discussion-and-feedback)  
[4. Resources](#4.-Resources)  

## 1. Introduction

Todo

## 2. Methods & results

### 2.1 Data collection

In [8]:
import pandas as pd
from IPython.display import display

demographic = pd.read_csv("data/demographic.csv")
products = pd.read_csv("data/product.csv")
transactions = pd.read_csv("data/transaction_data.csv")

In [7]:
display(demographic.head())
display(products.head())
display(transactions.head())

,AGE_DESC,MARITAL_STATUS_CODE,INCOME_DESC,HOMEOWNER_DESC,HH_COMP_DESC,HOUSEHOLD_SIZE_DESC,KID_CATEGORY_DESC,household_key
0,65+,A,35-49K,Homeowner,2 Adults No Kids,2,None/Unknown,1
1,45-54,A,50-74K,Homeowner,2 Adults No Kids,2,None/Unknown,7
2,25-34,U,25-34K,Unknown,2 Adults Kids,3,1,8
3,25-34,U,75-99K,Homeowner,2 Adults Kids,4,2,13
4,45-54,B,50-74K,Homeowner,Single Female,1,None/Unknown,16


,PRODUCT_ID,MANUFACTURER,DEPARTMENT,BRAND,COMMODITY_DESC,SUB_COMMODITY_DESC,CURR_SIZE_OF_PRODUCT
0,25671,2,GROCERY,National,FRZN ICE,ICE - CRUSHED/CUBED,22 LB
1,26081,2,MISC. TRANS.,National,NO COMMODITY DESCRIPTION,NO SUBCOMMODITY DESCRIPTION,
2,26093,69,PASTRY,Private,BREAD,BREAD:ITALIAN/FRENCH,
3,26190,69,GROCERY,Private,FRUIT - SHELF STABLE,APPLE SAUCE,50 OZ
4,26355,69,GROCERY,Private,COOKIES/CONES,SPECIALTY COOKIES,14 OZ


,household_key,BASKET_ID,DAY,PRODUCT_ID,QUANTITY,SALES_VALUE,STORE_ID,RETAIL_DISC,TRANS_TIME,WEEK_NO,COUPON_DISC,COUPON_MATCH_DISC
0,2375,26984851472,1,1004906,1,1.39,364,-0.60,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.00,1631,1,0.0,0.0
2,2375,26984851472,1,1036325,1,0.99,364,-0.30,1631,1,0.0,0.0
3,2375,26984851472,1,1082185,1,1.21,364,0.00,1631,1,0.0,0.0
4,2375,26984851472,1,8160430,1,1.50,364,-0.39,1631,1,0.0,0.0


### 2.2 Data cleaning and pre-processing

### 2.3 Descriptive Analysis

#### 2.3.2 Products offered

#### 2.3.3 Best selling product overall

### 2.4 Predictive Models

#### 2.4.1 ARIMA

#### 2.4.2 Random Forests

## 3. Discussion and feedback

## 4. Resources